In [1]:
import imaplib
import email
from email.header import decode_header
import webbrowser
import os
import traceback
import base64
from bs4 import BeautifulSoup
import config
import functions
import re

In [2]:
def connection():
    mail_pass = config.mail_pass
    username = config.username
    imap_server = "imap.mail.ru"
    imap = imaplib.IMAP4_SSL(imap_server)
    sts, res = imap.login(username, mail_pass)
    print(res)
    if sts == "OK":
        return imap
    else:
        return False


def encoded_words_to_text(encoded_words):
    # encoded_words=encoded_words.replace("\n", "").replace("\r", "").replace("\t", "").replace(" ", "")
    encoded_word_regex = r"=\?{1}(.+)\?{1}([B|Q])\?{1}(.+)\?{1}="
    charset, encoding, encoded_text = re.match(
        encoded_word_regex, encoded_words
    ).groups()
    if encoding == "B":
        byte_string = base64.b64decode(encoded_text)
    elif encoding == "Q":
        byte_string = quopri.decodestring(encoded_text)
    return byte_string.decode(charset)


def get_attaches(msg):
    attaches = list()
    for part in msg.walk():
        if part.get_content_disposition() == "attachment":
            if "pdf" in part.get_content_type():
                attaches.append(pdf_to_str(part.get_payload(decode=True)))
    if attaches:
        return attaches
    else:
        return [False]


def date_parse(msg_date):
    dt_obj = "".join(str(msg_date[:6]))
    dt_obj = dt_obj.strip("'(),")
    dt_obj = datetime.strptime(dt_obj, "%Y, %m, %d, %H, %M, %S")
    return dt_obj


def pdf_to_str(pdf):
    all_doc = ""
    try:
        with io.BytesIO(pdf) as open_pdf_file:
            read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
            num_pages = read_pdf.getNumPages()
            for page in read_pdf.pages:
                page_content = page
                all_doc += page_content.extractText()
            return all_doc
    except:
        return False

async def send_message(bot_token, message, chat, rpl=None, prv=None):
    bot = Bot(token=bot_token)
    await bot.get_session()
    obj = await bot.send_message(
        chat_id=chat,
        text=message,
        parse_mode="HTML",
        reply_to_message_id=rpl,
        disable_web_page_preview=prv,
    )
    await bot._session.close()
    return obj.message_id

def get_letter_text(body):   
    soup = BeautifulSoup(body, 'html.parser')
    paragraphs = soup.find_all('div')
    text=""
    for paragraph in paragraphs:
        text += paragraph.text + "\n"    
    return text.replace('\xa0', ' ')

def get_text_from_multipart(msg):
    payload = msg.get_payload()[0]
    print(payload["Content-Type"])
    if "text/plain" in payload["Content-Type"]:
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text = letter_text.replace("<","").replace(">","")       
    if "text/html" in payload["Content-Type"]:
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text=get_letter_text(letter_text)
    if "multipart" in payload["Content-Type"]:
        letter_text = (
            (
                base64.b64decode(
                    payload.get_payload()[0].get_payload()
                ).decode()
            )
            .lstrip()
            .rstrip()
        )
    return letter_text

def post_construct(msg_subj, msg_from, msg_email, letter_text, att_files):
    txt = ""
    txt += (
        "<b>"
        + str(msg_subj)
        + "</b>"
        + "\n\n"
        + str(msg_from)
        + "\n"
        + msg_email
        + "\n"
        + letter_text
        + "\n"
        + "вложения pdf: "
        + str(att_files)
    )
    return txt

In [202]:
imap = connection()

[b'Authentication successful']


In [221]:
status, messages = imap.select("INBOX")  # папка входящие
res, unseen_msg = imap.uid('search', "UNSEEN", "ALL")
unseen_msg = unseen_msg[0].decode("utf-8").split(" ")

In [222]:
if unseen_msg[0]:
    print(unseen_msg)
else:
    print('there are no unseen messages')

['26']


In [223]:
for letter in unseen_msg:
    print(letter)
    attachments=[]
    res, msg = imap.uid('fetch', letter, '(RFC822)')
    msg = email.message_from_bytes(msg[0][1]) # получаем объект email
    if msg.is_multipart():
        print('multipart')                          
    else:
        print('single')                

26
multipart


In [136]:
letter=str(24)
res, msg = imap.uid('fetch', letter, '(RFC822)')
msg = email.message_from_bytes(msg[0][1]) # получаем объект email

In [47]:
from email.policy import default as default_policy
res, msg = imap.uid('fetch', letter, '(RFC822)')
msg = email.message_from_bytes(msg[0][1], policy=default_policy)
body=msg.get_body('plain')
for row in body:
    print(row)

Content-Type
Content-Transfer-Encoding


In [48]:
res, msg_0 = imap.uid('fetch', letter, '(RFC822)')
msg_0[0][0] # Что это за цифры? 15814
b'19 (UID 28 RFC822 {110401}'

b'19 (UID 28 RFC822 {110401}'

In [230]:
#multipart
payload = msg.get_payload() #список, состоит из нескольких частей

In [231]:
payload

In [51]:
def get_text_from_multipart(payload):
    #payload = msg.get_payload()    
    print(payload["Content-Type"])
    if payload.is_multipart():
        print('m')
        letter_text = (
            (
                base64.b64decode(
                    payload.get_payload()[0].get_payload()
                ).decode()
            )
            .lstrip()
            .rstrip()
        )
        
    elif "text/plain" in payload["Content-Type"]:
        print('i')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )
        letter_text = letter_text.replace("<","").replace(">","")
        
    
    elif "text/html" in payload["Content-Type"]:
        print('ii')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )        
        letter_text=get_letter_text(letter_text)                     
    return letter_text

In [31]:
def encode_att_names(str_pl):
    enode_name=re.findall('\=\?.*?\?\=', str_pl)        
    if len(enode_name) == 1:            
        decode_name=(decode_header(enode_name[0])[0][0])            
        decode_name=decode_name.decode("utf-8")            
        str_pl=str_pl.replace(enode_name[0], decode_name)
    if len(enode_name) > 1:             
        nm=''
        for part in enode_name:                
            decode_name=(decode_header(part)[0][0])                        
            decode_name=decode_name.decode("utf-8")
            nm+=decode_name            
        str_pl=str_pl.replace(enode_name[0], nm)    
        for c, i in enumerate(enode_name):           
            if c > 0:                
                str_pl=str_pl.replace(enode_name[c], "").replace("\"",'').rstrip()
    return str_pl  

In [232]:
attachments=list()  
for pl in payload:
    
    print(pl.is_multipart())
    print('content_disposition', pl.get_content_disposition(), pl.get_content_subtype())
    print(pl["Content-Type"])
    
    if pl.is_multipart():                
        next_level=pl.get_payload()
        types=[]
        for level in next_level:
            print(type(level["Content-Type"]))            
            if len(types) == 0: #чтобы два раза не собиралась письмо                
                print(get_text_from_multipart(level))
            hl=level["Content-Type"]
            types.append(hl)                  
    if 'name' in pl["Content-Type"] and pl.get_content_disposition() == 'attachment':               
        str_pl=pl["Content-Type"]               
        str_pl=encode_att_names(str_pl)
        attachments.append(str_pl)
        print(str_pl)
    else:
        pass
        print(pl["Content-Type"])

False
content_disposition None html
text/html; charset=utf-8
text/html; charset=utf-8
False
content_disposition attachment pdf
application/pdf;
	name="=?UTF-8?B?0JDQvdC+0L3RgSBfINC/0LXRgtCw0L3Qui5wZGY=?="
application/pdf;
	name="Анонс _ петанк.pdf"


In [233]:
attachments

['application/pdf;\r\n\tname="Анонс _ петанк.pdf"']

In [56]:
def get_attachments():
    payload = msg.get_payload()
    attachments=list()  
    for pl in payload:
        if 'name' in pl["Content-Type"] and pl.get_content_disposition() == 'attachment':               
            str_pl=pl["Content-Type"]               
            str_pl=encode_att_names(str_pl)
            attachments.append(str_pl)
    return attachments

In [59]:
att_txt=""
for atts in attachments:
    att_txt+=(atts.strip()+"\n")

    

In [60]:
att_txt

'application/pdf; name="Бланк согласие на грипп.pdf"\n'

In [178]:
def decode_text(payload):    
    if payload.get_content_subtype() == 'plain':    
        print('i')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )        
        letter_text = letter_text.replace("<","").replace(">","").replace("\xa0","")
        return letter_text
    if payload.get_content_subtype() == 'html':   
        print('ii')
        letter_text = (
            (base64.b64decode(payload.get_payload()).decode())
            .lstrip()
            .rstrip()
        )        
        letter_text=get_letter_text(letter_text)
        return letter_text
        
    

In [219]:
def get_text_from_multipart(payload):
    for pl in payload: 
        count=0
        if pl.is_multipart():            
            level = pl.get_payload()        
            for part in level:
                if part.get_content_maintype() == 'text' and count == 0:         
                    letter_text = decode_text(part)
                    count += 1                                        
        else:        
            if pl.get_content_maintype() == 'text' and count == 0:           
                letter_text = decode_text(pl)                     
        return letter_text

In [247]:
get_text_from_multipart(payload)

ii


'Добрый вечер! 16 сентября на набережной Казанки состоится турнир по петанку. Турнир станет частью масштабной программы Дней парков и скверов, закрывающей сезон летних мероприятий. Подробнее — в пресс-релизе. \nДобрый вечер!\n \n16 сентября на набережной Казанки состоится турнир по петанку. Турнир станет частью масштабной программы Дней парков и скверов, закрывающей сезон летних мероприятий. Подробнее — в пресс-релизе.\n \n-- \nКонтакты для связи:\nДиляра Галеева, \nпресс-секретарь Наталии Фишман-Бекмамбетовой,\nруководитель пресс-службы Фонда "Институт развития городов РТ"\n+79172778334\n \n'

In [255]:
for pl in payload: 
    count=0
    if pl.is_multipart():            
        level = pl.get_payload()        
        for part in level:
            if part.get_content_maintype() == 'text' and count == 0:         
                letter_text = decode_text(part)
                count += 1                                        
    else:        
        if pl.get_content_maintype() == 'text' and count == 0:
            print(base64.b64decode(pl.get_payload()).decode())
            letter_text = decode_text(pl)                     
    

<div><div>Добрый вечер!</div><div> </div><div>16 сентября на набережной Казанки состоится турнир по петанку. Турнир станет частью масштабной программы Дней парков и скверов, закрывающей сезон летних мероприятий. Подробнее — в пресс-релизе.</div><div> </div></div><div><strong>-- </strong></div><div style="background-color:rgb( 255 , 255 , 255 );color:rgb( 0 , 0 , 0 );font-family:'calibri' , sans-serif;font-size:11pt;font-style:normal;font-weight:400;line-height:normal;margin:0cm;text-decoration-style:initial;text-indent:0px;text-transform:none;white-space:normal;word-spacing:0px"><strong><span style="color:black;font-size:12pt">Контакты для связи:</span></strong></div><div style="background-color:rgb( 255 , 255 , 255 );color:rgb( 0 , 0 , 0 );font-family:'calibri' , sans-serif;font-size:11pt;font-style:normal;font-weight:400;line-height:normal;margin:0cm;text-decoration-style:initial;text-indent:0px;text-transform:none;white-space:normal;word-spacing:0px"><span style="color:black;font-si

In [252]:
letter_text

'Добрый вечер! 16 сентября на набережной Казанки состоится турнир по петанку. Турнир станет частью масштабной программы Дней парков и скверов, закрывающей сезон летних мероприятий. Подробнее — в пресс-релизе. \nДобрый вечер!\n \n16 сентября на набережной Казанки состоится турнир по петанку. Турнир станет частью масштабной программы Дней парков и скверов, закрывающей сезон летних мероприятий. Подробнее — в пресс-релизе.\n \n-- \nКонтакты для связи:\nДиляра Галеева, \nпресс-секретарь Наталии Фишман-Бекмамбетовой,\nруководитель пресс-службы Фонда "Институт развития городов РТ"\n+79172778334\n \n'

In [119]:
#текст письма
for pl in payload:    
    print(pl.is_multipart())
    print('content main sub ', pl.get_content_maintype(), pl.get_content_subtype())
    print(pl["Content-Type"])
    
    if pl.is_multipart():                
        next_level=pl.get_payload()
        types=[]
        for level in next_level:
            print(type(level["Content-Type"]))
            print('2 main sub ', level.get_content_maintype(), level.get_content_subtype())
            if len(types) == 0: #чтобы два раза не собиралась письмо                
                print(get_text_from_multipart(level))
            hl=level["Content-Type"]
            types.append(hl)                  
    if 'name' in pl["Content-Type"] and pl.get_content_disposition() == 'attachment':               
        str_pl=pl["Content-Type"]               
        str_pl=encode_att_names(str_pl)
        attachments.append(str_pl)
        print(str_pl)
    else:
        pass
        print(pl["Content-Type"])

True
content main sub  multipart alternative
multipart/alternative;
	boundary="--ALT--c9B7c55f14c90A9F252A4b12Ff8E28d21662632529"
<class 'str'>
2 main sub  text plain
Content-Type


AttributeError: 'str' object has no attribute 'is_multipart'

In [130]:
letter=base64.b64decode(payload).decode()

In [131]:
letter

'<div><div><strong>Мошенники позвали жителей Нижнекамска на кастинг в массовку новой экранизации Кира Булычева</strong></div></div><div>\xa0</div><div><div>В одном из пабликов Нижнекамска появился пост о набор массовки для фильма «Сто лет тому вперёд». Соискателей просят отправлять личные данные на электронную почту.</div><div>\xa0</div><div>Поиск по соцсети «ВКонтакте» показал, что точно такие же посты появились в группах Анапы, Геленджика, Новороссийска, Москвы, Тюмени. Пользователи, которые отправили заявки на почту, пишут, что их якобы утвердили на роль и попросили перевести деньги на изготовление «индивидуальной карты актёра».</div><div>\xa0</div><div>НТР 24 обратился в кинокомпанию «Водород», которая, согласно данным из открытых источником, занимается производством фильма. Там заявили, что не размещали таких объявлений.</div><div>\xa0</div><div>«Со своей стороны, Кинокомпания "Водород" намерена предпринять меры, направленные на пресечение подобного рода мошеннических действий».</